# Main imports and code - Preparation

In [1]:
# check which gpu we're using
!nvidia-smi

Mon Mar  4 22:22:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install simpletransformers
!pip install tensorboardx
!pip install googletrans
!pip install nltk
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.4 

In [3]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval

In [4]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

Cuda available?  True


In [5]:
if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


## Fetch Don't Patronize Me! data manager module

In [6]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [7]:
#the raw dataset
!wget -O dontpatronizeme_pcl.tsv https://raw.githubusercontent.com/CRLala/NLPLabs-2024/main/Dont_Patronize_Me_Trainingset/dontpatronizeme_pcl.tsv

--2024-03-04 22:23:39--  https://raw.githubusercontent.com/CRLala/NLPLabs-2024/main/Dont_Patronize_Me_Trainingset/dontpatronizeme_pcl.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3122842 (3.0M) [text/plain]
Saving to: ‘dontpatronizeme_pcl.tsv’

dontpatronizeme_pcl 100%[===================>]   2.98M  --.-KB/s    in 0.07s   

2024-03-04 22:23:39 (45.3 MB/s) - ‘dontpatronizeme_pcl.tsv’ saved [3122842/3122842]



In [8]:
# their training selection set - i assume it's for picking up corresponding id from the raw dataset
!wget -O train_semeval_parids-labels.csv https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv

--2024-03-04 22:23:39--  https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241839 (236K) [text/plain]
Saving to: ‘train_semeval_parids-labels.csv’

train_semeval_parid 100%[===================>] 236.17K  --.-KB/s    in 0.03s   

2024-03-04 22:23:39 (8.73 MB/s) - ‘train_semeval_parids-labels.csv’ saved [241839/241839]



In [9]:
#their testing selection set
!wget -O dev_semeval_parids-labels.csv https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv

--2024-03-04 22:23:40--  https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61151 (60K) [text/plain]
Saving to: ‘dev_semeval_parids-labels.csv’

dev_semeval_parids- 100%[===================>]  59.72K  --.-KB/s    in 0.01s   

2024-03-04 22:23:40 (5.02 MB/s) - ‘dev_semeval_parids-labels.csv’ saved [61151/61151]



In [10]:
#testing set
!wget -O task4_test.tsv https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/TEST/task4_test.tsv

--2024-03-04 22:23:40--  https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/TEST/task4_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145277 (1.1M) [text/plain]
Saving to: ‘task4_test.tsv’

task4_test.tsv      100%[===================>]   1.09M  --.-KB/s    in 0.05s   

2024-03-04 22:23:40 (20.7 MB/s) - ‘task4_test.tsv’ saved [1145277/1145277]



In [11]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [12]:
from dont_patronize_me import DontPatronizeMe

In [13]:
dpm = DontPatronizeMe('.', 'task4_test.tsv')

In [14]:
dpm.load_task1()
#Paragraphs with original labels of 0 or 1 are considered to be negative examples of PCL and will have the label 0 = negative.
#Paragraphs with original labels of 2, 3 or 4 are considered to be positive examples of PCL and will have the label 1 = positive.

## Load paragraph IDs

In [15]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

In [16]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

In [17]:
data=dpm.train_task1_df

In [18]:
data


,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""White House press secretary Sean Spicer said ...",0,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" Just like we received migrants fleeing El ...",0,0
...,...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,"""Sri Lankan norms and culture inhibit women fr...",0,1
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0,0
10466,10467,@@20282330,in-need,ng,""""""" She has one huge platform , and informatio...",1,3
10467,10468,@@16753236,hopeless,in,""""""" Anja Ringgren Loven I ca n't find a word t...",1,4


In [19]:
data = data.dropna()

# RAW Dataset - Q1.a & b Data Analysis Code - SAVE FOR MICHAEL

In [20]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# label_distribution = data['label'].value_counts()
# label_distribution.index = label_distribution.index.astype(str)
# orig_label_distribution = data['orig_label'].value_counts()
# total = label_distribution.sum()

# # distribution of 'label'
# plt.figure(figsize=(3.5, 0.75))
# barplot = sns.barplot(x=label_distribution.values, y=label_distribution.index, orient='h', color='lightblue')
# plt.title('')
# plt.xlabel('')
# plt.ylabel('')
# plt.xticks([])
# for p in barplot.patches:
#     width = p.get_width()
#     count = int(width)
#     percentage = f"({count / total:.1%})"
#     plt.text(width + 1, p.get_y() + p.get_height() / 2, f"{count} {percentage}", ha='left', va='center', color='black')
# for spine in plt.gca().spines.values():
#     spine.set_visible(False)
# plt.gca().tick_params(left=False)  # Remove y-axis tick marks
# plt.tight_layout()  # Adjust the layout to make sure everything fits without overlapping
# plt.savefig('distribution_of_pcl.pdf', format='pdf', bbox_inches='tight')  # Save the figure as a PDF
# plt.show()

# # distribution of 'orig_label'
# agg_data = data.groupby('orig_label').agg(
#     count=('orig_label', 'size'),
#     most_common_label=('label', lambda x: x.mode()[0])
# ).reset_index()
# agg_data['label_group'] = agg_data['most_common_label'].map({0: 'No PCL', 1: 'PCL'})
# color_map = {'No PCL': 'lightblue', 'PCL': 'salmon'}
# plt.figure(figsize=(3.5, 3.5))
# sns.barplot(x='orig_label', y='count', hue='label_group', dodge=False,
#             palette=color_map, data=agg_data)
# plt.title('Distribution of Original Labels')
# plt.xlabel('Original Label')
# plt.ylabel('')
# legend = plt.legend(title='')
# for spine in plt.gca().spines.values():
#     spine.set_visible(False)
# plt.tick_params(axis='x', which='both', bottom=False, top=False)
# legend.get_frame().set_edgecolor('none')  # This removes the border
# plt.tight_layout()
# plt.savefig('distribution_of_orig_labels_by_group.pdf', format='pdf', bbox_inches='tight')
# plt.show()

# # PCL by vulnerable group
# keyword_label_counts = data.groupby(['keyword', 'label']).size().unstack(fill_value=0)
# keyword_label_proportions = keyword_label_counts.div(keyword_label_counts.sum(axis=1), axis=0)
# keyword_label_proportions.reset_index(inplace=True)
# fig, ax = plt.subplots(figsize=(3.5, 2.75))
# ax.barh(keyword_label_proportions['keyword'], keyword_label_proportions[0], label='No PCL', color='lightblue')
# ax.barh(keyword_label_proportions['keyword'], keyword_label_proportions[1], left=keyword_label_proportions[0], label='PCL', color='salmon')
# ax.set_ylabel('')
# ax.set_xlabel('Proportion')
# ax.set_title('')
# ax.tick_params(axis='y', which='both', left=False)  # Remove y-axis tick marks
# legend = ax.legend(title='', loc='lower center', bbox_to_anchor=(0.5, 1), ncol=2)
# legend.get_frame().set_linewidth(0.0)  # Remove the legend border
# for spine in ax.spines.values():
#     spine.set_visible(False)
# plt.tight_layout()
# plt.savefig('pcl_by_keyword.pdf', format='pdf', bbox_inches='tight')
# plt.show()

# # Box plot of paragraph length by PCL label
# data['text_length'] = data['text'].apply(len)
# data['label_str'] = data['label'].map({0: 'No PCL', 1: 'PCL'})
# palette_colors = {'No PCL': 'lightblue', 'PCL': 'salmon'}
# plt.figure(figsize=(3.5, 2.75))  # Adjust the figure size as needed
# ax = sns.boxplot(x='text_length', y='label_str', hue='label_str', data=data, orient='h',
#                  palette={'No PCL': 'lightblue', 'PCL': 'salmon'}, dodge=False, legend=False)
# plt.title('')
# plt.xlabel('Text length')
# plt.ylabel('')
# for spine in ax.spines.values():
#     spine.set_visible(False)
# ax.set_xlim(0, 2000)
# ax.tick_params(axis='y', which='both', left=False)  # Remove y-axis tick marks
# plt.tight_layout()
# plt.savefig('pcl_by_length.pdf', format='pdf', bbox_inches='tight')
# plt.show()



## Get the raw training set (Task 1) - without split

In [21]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })


In [22]:
import random

In [23]:
trdf1 = pd.DataFrame(rows)

In [24]:
trdf1

,par_id,community,text,label
0,4341,poor-families,"The scheme saw an estimated 150,000 children f...",1
1,4136,homeless,Durban 's homeless communities reconciliation ...,1
2,10352,poor-families,The next immediate problem that cropped up was...,1
3,8279,vulnerable,Far more important than the implications for t...,1
4,1164,poor-families,To strengthen child-sensitive social protectio...,1
...,...,...,...,...
8370,8380,refugee,Rescue teams search for survivors on the rubbl...,0
8371,8381,hopeless,The launch of ' Happy Birthday ' took place la...,0
8372,8382,homeless,"The unrest has left at least 20,000 people dea...",0
8373,8383,hopeless,You have to see it from my perspective . I may...,0


In [25]:
trdf1.to_csv("trdf1.csv")

## get the raw test set (Task 1)

In [26]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })


In [27]:
len(rows)

2094

In [28]:
tedf1 = pd.DataFrame(rows)
#print(tedf1)

In [29]:
random.shuffle(tedf1.text.tolist())
tedf1.to_csv("tedf1.csv")
#shuffled_tedf1 = tedf1.sample(frac=1).reset_index(drop=True)

In [30]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix


def evaluate_model(true_labels, preds):
    f1 = f1_score(true_labels, preds)
    accuracy = accuracy_score(true_labels, preds)
    precision = precision_score(true_labels, preds)
    recall = recall_score(true_labels, preds)
    conf_matrix = confusion_matrix(true_labels, preds)
    print(f"F1 Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")


    print("Confusion Matrix:")
    print(conf_matrix)



## Baseline for Task 1

In [31]:
print(len(trdf1[trdf1.label==0]))
print(len(trdf1[trdf1.label==1]))
print((len(trdf1[trdf1.label==0]))/len(trdf1[trdf1.label==1]))

7581
794
9.547858942065492


Neg is 10 times more than positive class.

### RoBERTa Negative : Positive = 2:1

- non-pcl : pcl = 2:1
- 46.7% F1 on offical dev set

In [32]:
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)
#0, limited to twice the number of positive examples
training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])

In [33]:
# from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs

# results = []

# for seed in [random.randint(1, 100) for _ in range(10)]:
#     for base in ['roberta', 'deberta']:
#         baseline = base

#         task1_model_args = ClassificationArgs(num_train_epochs=1,
#                                             no_save=True,
#                                             no_cache=True,
#                                             manual_seed = seed,
#                                             overwrite_output_dir=True,
#                                             use_multiprocessing=False,
#                                             use_multiprocessing_for_evaluation=False)
#         if baseline == 'roberta':
#             task1_model = ClassificationModel("roberta",
#                                             'roberta-base',
#                                             args = task1_model_args,
#                                             num_labels=2,
#                                             use_cuda=cuda_available)
#         elif baseline == 'deberta':
#             task1_model = ClassificationModel("deberta",
#                                             'microsoft/deberta-base',
#                                             args = task1_model_args,
#                                             num_labels=2,
#                                             use_cuda=cuda_available)
#         else:
#             print("Please enter valid model to take as a baseline")
#         task1_model.train_model(training_set1[['text', 'label']])
#         preds_task1, _ = task1_model.predict(tedf1.text.tolist())
#         true_labels = tedf1.label.tolist()
#         f1 = f1_score(true_labels, preds_task1)
#         results_tuple = (base, seed, f1)
#         results.append(results_tuple)
# results = pd.DataFrame(results, columns=['Model', 'Seed', 'F1'])
# Creating boxplot
# plt.figure(figsize=(3.5, 2.75))
# ax = sns.boxplot(x='F1', y='Model', hue='Model', data=results, dodge=False, legend=False,
#             palette={'roberta': 'lightblue', 'deberta': 'salmon'})
# plt.title('')
# plt.xlabel('F1 Score')
# plt.ylabel('')
# plt.tight_layout()
# for spine in ax.spines.values():
#     spine.set_visible(False)
# ax.tick_params(axis='y', which='both', left=False)  # Remove y-axis tick marks
# plt.savefig('avg_berta_performance.pdf', format='pdf', bbox_inches='tight')
# plt.show()
# torch.cuda.memory_allocated()
# torch.cuda.empty_cache()
# del task1_model
# del preds_task1
# del true_labels
# torch.cuda.memory_allocated()

### <strike>{RoBERTa Negative : Positive = 3:1}</strike>
- change ratio to 3:1
- F1: 52.21%

In [34]:
# training_set2 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*3]])

# # train model
# task1_model.train_model(training_set2[['text', 'label']])
# # run predictions
# preds_task1, _ = task1_model.predict(tedf1.text.tolist())
# true_labels = tedf1.label.tolist()
# evaluate_model(true_labels, preds_task1)

# Get Train + Val + Test Set

In [35]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(trdf1, test_size=0.2, random_state=905)
test_set = tedf1

In [36]:
print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

(6700, 4)
(1675, 4)
(2094, 4)


In [37]:
# model_args = ClassificationArgs(
#           num_train_epochs = 4,
#           learning_rate = 2e-5,
#           overwrite_output_dir = True,
#           evaluate_during_training = True,
#           manual_seed = 42,
#           use_early_stopping = True,
#           early_stopping_consider_epochs = True,
#           early_stopping_patience = 3,
#           save_steps = -1 , # Save the model only at the end of training
#           train_batch_size = 16,
#           eval_batch_size = 8
# )


# model = ClassificationModel('electra', 'google/electra-small-discriminator', args=model_args,num_labels=2,
#                                   use_cuda=cuda_available)


# model.train_model(train_set[['text', 'label']], eval_df = val_set[['text', 'label']])

# predictions, raw_outputs = model.predict(test_set['text'].tolist())
# evaluate_model(test_set['label'].tolist(),predictions )

# Q2.a & b

## Deberta - on our splitted dataset

- 57.36%

In [38]:
# from google.colab import drive
# drive.mount('/content/drive')

In [39]:
# deberta_model_args = ClassificationArgs(
#     num_train_epochs = 4,
#     learning_rate=2e-5,
#     overwrite_output_dir=True,
#     evaluate_during_training=True,
#     manual_seed=42,
#     use_early_stopping=True,
#     early_stopping_consider_epochs=True,
#     early_stopping_patience=3,
#     save_steps=-1,  # Save the model only at the end of training
#     train_batch_size=32,
#     eval_batch_size=16,
#     output_dir='./content/drive/My Drive/my_deberta_model'
# )

# deberta_model = ClassificationModel(
#     'deberta',
#     'microsoft/deberta-base',
#     args=deberta_model_args,
#     num_labels=2,
#     use_cuda=cuda_available
#     )


# deberta_model.train_model(train_set[['text', 'label']], eval_df=val_set[['text', 'label']])


# result, model_outputs, wrong_predictions = deberta_model.eval_model(val_set[['text', 'label']])


# print(result)

# predictions_deberta, raw_outputs = deberta_model.predict(test_set['text'].tolist())
# evaluate_model(test_set['label'].tolist(),predictions_deberta )


### store

In [40]:
# labels2file([[k] for k in predictions_deberta], 'dev.txt')

In [41]:
# dpm.load_test()
# test_df = dpm.test_set_df

In [42]:
# predictions, raw_outputs = deberta_model.predict(test_df['text'].tolist())
# labels2file([[k] for k in predictions], 'test.txt')

In [43]:
# torch.cuda.memory_allocated()
# torch.cuda.empty_cache()
# del deberta_model
# del predictions_deberta
# torch.cuda.memory_allocated()

## Deberta with Scheduler - refer to another notebook

In [44]:
# from transformers import DebertaTokenizer, DebertaForSequenceClassification, TrainingArguments, Trainer
# from transformers import get_linear_schedule_with_warmup
# import torch
# import Dataset

# tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
# model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=2)

# # Tokenize the dataset
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# train_dataset = Dataset.from_pandas(train_set)
# val_dataset = Dataset.from_pandas(val_set)
# #model = DataParallel(model).cuda()

# # # Tokenize your dataset here
# def tokenize_function(example):
#     return tokenizer(example["text"], truncation=True, padding="max_length")

# tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
# tokenized_eval_dataset = val_dataset.map(tokenize_function, batched=True)

# tokenized_train_dataset = tokenized_train_dataset.with_format("torch")
# tokenized_eval_dataset = tokenized_eval_dataset.with_format("torch")

# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=4,
#     per_device_train_batch_size=32,
#     per_device_eval_batch_size=16,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir='./logs',
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
# )
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,

# )

# # Define custom optimizer and scheduler
# optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
# num_train_steps = len(trainer.train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=num_train_steps)

# # Pass optimizer and scheduler to trainer
# trainer.optimizer = optimizer
# trainer.lr_scheduler = scheduler

# # Train model
# trainer.train()


In [45]:
# import numpy as np

# test_dataset = Dataset.from_pandas(test_set)
# tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
# tokenized_test_dataset = tokenized_test_dataset.with_format("torch")

# #predictions_deberta, raw_outputs = trainer.predict(test_set['text'].tolist())
# predictions_deberta = trainer.predict(tokenized_test_dataset)

# # Extract the predictions (logits) and true labels
# pred_logits = predictions_deberta.predictions
# true_labels = predictions_deberta.label_ids

# # Convert logits to predicted class indices
# pred_labels = np.argmax(pred_logits, axis=1)

# # Evaluate the model using your custom evaluation function
# evaluate_model(true_labels, pred_labels)
# #evaluate_model(tokenized_test_dataset,predictions_deberta)

## <strike>Roberta - on our own splitted dataset</strike>

In [47]:
# roberta_model_args = ClassificationArgs(
#     num_train_epochs = 4,
#     learning_rate=2e-5,
#     overwrite_output_dir=True,
#     evaluate_during_training=True,
#     manual_seed=42,
#     use_early_stopping=True,
#     early_stopping_consider_epochs=True,
#     early_stopping_patience=3,
#     save_steps=-1,  # Save the model only at the end of training
#     train_batch_size=16,
#     eval_batch_size=8,
# )

# roberta_model = ClassificationModel(
#     'roberta',
#     'roberta-base',
#     args=roberta_model_args,
#     num_labels=2,
#     use_cuda=cuda_available
#     )


# roberta_model.train_model(train_set[['text', 'label']], eval_df=val_set[['text', 'label']])


# result, model_outputs, wrong_predictions = roberta_model.eval_model(val_set[['text', 'label']])


# print(result)

# predictions_roberta, raw_outputs = roberta_model.predict(test_set['text'].tolist())
# evaluate_model(test_set['label'].tolist(),predictions_roberta )


# Q2.d - before big clean
- these data are primarily cleaned during implementation - tokenized, and stop word

## BOW

tokenize words of length 3 to 10, use unigrams (ngram_range=(1, 1)), limit the features to the 23,000 most frequent words, and exclude English stop words.

Training: The Logistic Regression model is trained with a very high regularization strength (C=1500) and a high iteration limit to ensure convergence.

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
#reference: https://www.kaggle.com/code/ziedbaklouti/bow-logistic-regression-sklearn?scriptVersionId=33955168&cellId=5

# Preprocess and Vectorize Text Data
vect1 = CountVectorizer(token_pattern=r'\w{3,10}', ngram_range=(1, 1),
                        max_features=23000, stop_words='english')

# Fit the vectorizer on the training data and transform the training data
X_train_vect = vect1.fit_transform(train_set['text'])
# Transform the val data
X_val_vect = vect1.transform(val_set['text'])

#Train a Logistic Regression Model
log_reg1 = LogisticRegression(C=1500, max_iter=10000)

# Fit the model on the vectorized training data
log_reg1.fit(X_train_vect, train_set['label'])

#Make Predictions and Evaluate the Model
y_predicted_val = log_reg1.predict(X_val_vect)
print('------Val Set--------------')
evaluate_model(val_set['label'],y_predicted_val)

# Transform the test data
print('------Test Set--------------')
X_test_vect = vect1.transform(test_set['text'])
y_predicted_test = log_reg1.predict(X_test_vect)
evaluate_model(test_set['label'],y_predicted_test)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neverthele'] not in stop_words.
  warnings.warn(


------Val Set--------------
F1 Score: 0.2867647058823529
Accuracy: 0.8841791044776119
Precision: 0.3786407766990291
Recall: 0.23076923076923078
Confusion Matrix:
[[1442   64]
 [ 130   39]]
------Test Set--------------
F1 Score: 0.27826086956521745
Accuracy: 0.8810888252148997
Precision: 0.3287671232876712
Recall: 0.24120603015075376
Confusion Matrix:
[[1797   98]
 [ 151   48]]


In [49]:
import numpy as np

misclassified_indices = np.where(y_predicted_val != val_set['label'])[0]

# first misclassified example
if len(misclassified_indices) > 0:
    example_index = misclassified_indices[0]
    misclassified_text = val_set['text'].iloc[example_index]
    true_label = val_set['label'].iloc[example_index]
    predicted_label = y_predicted_val[example_index]

    print("Example of Misclassified Text:")
    print(f"Text: {misclassified_text}")
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
else:
    print("No misclassified examples found.")


Example of Misclassified Text:
Text: As a result , millions of Syrians as well as peoples of other countries in the region have been made both homeless and hopeless .
True Label: 1, Predicted Label: 0


## TF-IDF

uses unigrams (ngram_range=(1, 1)), which may not capture the sentiment expressed through phrases.

Max Features Limitation: Limiting to 1000 max features

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess and Vectorize Text Data using TF-IDF
vect1 = TfidfVectorizer(ngram_range=(1, 1),max_features=1000, stop_words='english').fit(train_set.text)

# Fit the vectorizer on the training data and transform the training data
X_train_vect = vect1.fit_transform(train_set['text'])
# Transform the validation data
X_val_vect = vect1.transform(val_set['text'])

# Train a Logistic Regression Model
log_reg1 = LogisticRegression(C=1500, max_iter=10000)

# Fit the model on the vectorized training data
log_reg1.fit(X_train_vect, train_set['label'])
print('------Val Set--------------')
# Make Predictions and Evaluate the Model on the validation set
y_predicted_val = log_reg1.predict(X_val_vect)
evaluate_model(val_set['label'],y_predicted_val)
print('------Test Set--------------')

X_test_vect = vect1.transform(test_set['text'])
y_predicted_test = log_reg1.predict(X_test_vect)
evaluate_model(test_set['label'],y_predicted_test)

------Val Set--------------
F1 Score: 0.2828282828282828
Accuracy: 0.8728358208955224
Precision: 0.328125
Recall: 0.2485207100591716
Confusion Matrix:
[[1420   86]
 [ 127   42]]
------Test Set--------------
F1 Score: 0.27419354838709675
Accuracy: 0.8710601719197708
Precision: 0.2947976878612717
Recall: 0.2562814070351759
Confusion Matrix:
[[1773  122]
 [ 148   51]]


In [51]:
misclassified_indices_val = np.where(y_predicted_val != val_set['label'])[0]

if len(misclassified_indices_val) > 0:
    example_index_val = misclassified_indices_val[0]
    misclassified_text_val = val_set['text'].iloc[example_index_val]
    true_label_val = val_set['label'].iloc[example_index_val]
    predicted_label_val = y_predicted_val[example_index_val]

    print("Example of Misclassified Text in Validation Set:")
    print(f"Text: {misclassified_text_val}")
    print(f"True Label: {true_label_val}, Predicted Label: {predicted_label_val}")
else:
    print("No misclassified examples found in the validation set.")


Example of Misclassified Text in Validation Set:
Text: As a result , millions of Syrians as well as peoples of other countries in the region have been made both homeless and hopeless .
True Label: 1, Predicted Label: 0


# Q2.c & e

## Pre-Processing

- Lowercasing the text.
- Removing punctuation and special characters
- Tokenizing the text.
- Removing stopwords.
- Stemming & Lemmatization.

In [52]:

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize

train_set_processed = train_set
val_set_processed = val_set
test_set_processed = test_set

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # lowercase
    text = text.lower()
    # punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # tokenize
    tokens = word_tokenize(text)
    # stop words
    tokens = [word for word in tokens if word not in stop_words]
    # stem & lemma
    #lemmatization is more context-aware and can better handle different forms
    # of a word to bring it to a meaningful base form. After lemmatization,
    # stemming can further reduce the word to its root form
    tokens = [stemmer.stem(lemmatizer.lemmatize(word)) for word in tokens]
    text = ' '.join(tokens)
    return text

train_set_processed['text'] = train_set_processed['text'].apply(preprocess_text)
val_set_processed['text'] = val_set_processed['text'].apply(preprocess_text)
test_set_processed['text'] = test_set_processed['text'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [53]:
train_set_processed['text']

1152    stakehold view violenc woman societi includ us...
157     nt want detract achiev paralympian hope peopl ...
5765    uk tri pretend someon els problem refuge migra...
3894    southeast louisiana buck nation trend becam le...
5285    lo angel ap man deport unit state six time exp...
                              ...                        
5235    project launch last week nungwi north district...
4509    mani refuge el salvador settl lo angel westlak...
883     care way societi chang upset mani peopl unfair...
3915    bernard also deni act unfairli target poor fam...
2246    pep guardiola manchest citi travel west ham li...
Name: text, Length: 6700, dtype: object

### Deberta after pre-processing

In [54]:
deberta_model_args = ClassificationArgs(
    num_train_epochs = 4,
    learning_rate=2e-5,
    overwrite_output_dir=True,
    evaluate_during_training=True,
    manual_seed=42,
    use_early_stopping=True,
    early_stopping_consider_epochs=True,
    early_stopping_patience=3,
    save_steps=-1,  # Save the model only at the end of training
    train_batch_size=32,
    eval_batch_size=16,
)

deberta_model = ClassificationModel(
    'deberta',
    'microsoft/deberta-base',
    args=deberta_model_args,
    num_labels=2,
    use_cuda=cuda_available
    )


deberta_model.train_model(train_set_processed[['text', 'label']], eval_df=val_set_processed[['text', 'label']])


result, model_outputs, wrong_predictions = deberta_model.eval_model(val_set_processed[['text', 'label']])


print(result)

predictions_deberta, raw_outputs = deberta_model.predict(test_set_processed['text'].tolist())
evaluate_model(test_set_processed['label'].tolist(),predictions_deberta )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/105 [00:00<?, ?it/s]

{'mcc': 0.4044296650574007, 'accuracy': 0.9020895522388059, 'f1_score': 0.4533333333333333, 'tp': 68, 'tn': 1443, 'fp': 63, 'fn': 101, 'auroc': 0.8548311684229551, 'auprc': 0.4719611667242102, 'eval_loss': 0.30094010217026584}


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/131 [00:00<?, ?it/s]

F1 Score: 0.448
Accuracy: 0.9011461318051576
Precision: 0.4772727272727273
Recall: 0.4221105527638191
Confusion Matrix:
[[1803   92]
 [ 115   84]]


In [55]:
# torch.cuda.memory_allocated()
# torch.cuda.empty_cache()
# del deberta_model
# del predictions_deberta
# torch.cuda.memory_allocated()

### BOW after pre-processing

tokenize words of length 3 to 10, use unigrams (ngram_range=(1, 1)), limit the features to the 23,000 most frequent words, and exclude English stop words.

Training: The Logistic Regression model is trained with a very high regularization strength (C=1500) and a high iteration limit to ensure convergence.

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
#reference: https://www.kaggle.com/code/ziedbaklouti/bow-logistic-regression-sklearn?scriptVersionId=33955168&cellId=5

# Preprocess and Vectorize Text Data
vect1 = CountVectorizer(token_pattern=r'\w{3,10}', ngram_range=(1, 1),
                        max_features=23000, stop_words='english')

# Fit the vectorizer on the training data and transform the training data
X_train_vect = vect1.fit_transform(train_set_processed['text'])
# Transform the val data
X_val_vect = vect1.transform(val_set_processed['text'])

#Train a Logistic Regression Model
log_reg1 = LogisticRegression(C=1500, max_iter=10000)

# Fit the model on the vectorized training data
log_reg1.fit(X_train_vect, train_set_processed['label'])

#Make Predictions and Evaluate the Model
y_predicted_val = log_reg1.predict(X_val_vect)
print('------Val Set--------------')
evaluate_model(val_set_processed['label'],y_predicted_val)

# Transform the test data
print('------Test Set--------------')
X_test_vect = vect1.transform(test_set['text'])
y_predicted_test = log_reg1.predict(X_test_vect)
evaluate_model(test_set['label'],y_predicted_test)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neverthele'] not in stop_words.
  warnings.warn(


------Val Set--------------
F1 Score: 0.28753993610223644
Accuracy: 0.8668656716417911
Precision: 0.3125
Recall: 0.26627218934911245
Confusion Matrix:
[[1407   99]
 [ 124   45]]
------Test Set--------------
F1 Score: 0.3202099737532808
Accuracy: 0.876313276026743
Precision: 0.33516483516483514
Recall: 0.3065326633165829
Confusion Matrix:
[[1774  121]
 [ 138   61]]


In [57]:
misclassified_indices = np.where(y_predicted_val != val_set_processed['label'])[0]

# first misclassified example
if len(misclassified_indices) > 0:
    example_index = misclassified_indices[0]
    misclassified_text = val_set_processed['text'].iloc[example_index]
    true_label = val_set_processed['label'].iloc[example_index]
    predicted_label = y_predicted_val[example_index]

    print("Example of Misclassified Text:")
    print(f"Text: {misclassified_text}")
    print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
else:
    print("No misclassified examples found.")


Example of Misclassified Text:
Text: result million syrian well peopl countri region made homeless hopeless
True Label: 1, Predicted Label: 0


### TD-IDF after pre-processing

uses unigrams (ngram_range=(1, 1)), which may not capture the sentiment expressed through phrases.

Max Features Limitation: Limiting to 1000 max features

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess and Vectorize Text Data using TF-IDF
vect1 = TfidfVectorizer(ngram_range=(1, 1),max_features=1000, stop_words='english').fit(train_set.text)

# Fit the vectorizer on the training data and transform the training data
X_train_vect = vect1.fit_transform(train_set_processed['text'])
# Transform the validation data
X_val_vect = vect1.transform(val_set_processed['text'])

# Train a Logistic Regression Model
log_reg1 = LogisticRegression(C=1500, max_iter=10000)

# Fit the model on the vectorized training data
log_reg1.fit(X_train_vect, train_set_processed['label'])
print('------Val Set--------------')
# Make Predictions and Evaluate the Model on the validation set
y_predicted_val = log_reg1.predict(X_val_vect)
evaluate_model(val_set_processed['label'],y_predicted_val)
print('------Test Set--------------')

X_test_vect = vect1.transform(test_set['text'])
y_predicted_test = log_reg1.predict(X_test_vect)
evaluate_model(test_set['label'],y_predicted_test)

------Val Set--------------
F1 Score: 0.3024691358024691
Accuracy: 0.8650746268656716
Precision: 0.3161290322580645
Recall: 0.28994082840236685
Confusion Matrix:
[[1400  106]
 [ 120   49]]
------Test Set--------------
F1 Score: 0.27518427518427513
Accuracy: 0.8591212989493792
Precision: 0.2692307692307692
Recall: 0.2814070351758794
Confusion Matrix:
[[1743  152]
 [ 143   56]]


In [59]:
import numpy as np

# Identifying indices of misclassified examples in the validation set
misclassified_indices_val = np.where(y_predicted_val != val_set_processed['label'])[0]

if len(misclassified_indices_val) > 0:
    example_index_val = misclassified_indices_val[0]  # Just choose the first misclassified example for analysis
    misclassified_text_val = val_set_processed['text'].iloc[example_index_val]
    true_label_val = val_set_processed['label'].iloc[example_index_val]
    predicted_label_val = y_predicted_val[example_index_val]

    print("Example of Misclassified Text in Validation Set:")
    print(f"Text: {misclassified_text_val}")
    print(f"True Label: {true_label_val}, Predicted Label: {predicted_label_val}")
else:
    print("No misclassified examples found in the validation set.")


Example of Misclassified Text in Validation Set:
Text: result million syrian well peopl countri region made homeless hopeless
True Label: 1, Predicted Label: 0


# Q3

# Prepare submission

In [60]:
!cat dev.txt | head -n 10

cat: dev.txt: No such file or directory


In [61]:
!zip submission.zip dev.txt test.txt

	zip warning: name not matched: dev.txt
	zip warning: name not matched: test.txt

zip error: Nothing to do! (submission.zip)


In [62]:
!cat test.txt | head -n 10

cat: test.txt: No such file or directory
